## Decision Tree Exercises

### Part 1

Using the titanic data, in your classification-exercises repository, create a notebook, decision_tree.ipynb where you will do the following:

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from pydataset import data

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, plot_tree

from acquire import get_iris_data
from acquire import get_titanic_data
from acquire import get_telco_data
from prepare import split_data
import os
import acquire
from env import get_db_url

In [37]:
titanic_df = get_titanic_data()
titanic_df

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,male,27.0,0,0,13.0000,S,Second,NaN,Southampton,1
887,887,1,1,female,19.0,0,0,30.0000,S,First,B,Southampton,1
888,888,0,3,female,NaN,1,2,23.4500,S,Third,NaN,Southampton,0
889,889,1,1,male,26.0,0,0,30.0000,C,First,C,Cherbourg,1


In [38]:
def clean_titanic(df):

    df = df.drop(columns =['embark_town','class','deck'])

    df.embarked = df.embarked.fillna(value='S')

    dummy_df = pd.get_dummies(df[['sex','embarked']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    return df

In [39]:
titanic_df = clean_titanic(titanic_df)

In [40]:
titanic_df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone,sex_male,embarked_Q,embarked_S
0,0,0,3,male,22.0,1,0,7.2500,S,0,1,0,1
1,1,1,1,female,38.0,1,0,71.2833,C,0,0,0,0
2,2,1,3,female,26.0,0,0,7.9250,S,1,0,0,1
3,3,1,1,female,35.0,1,0,53.1000,S,0,0,0,1
4,4,0,3,male,35.0,0,0,8.0500,S,1,1,0,1


In [41]:
train, validate, test = split_data(titanic_df, col_to_stratify='survived')
train.shape, validate.shape, test.shape

((534, 13), (178, 13), (179, 13))

In [42]:
X_train = train.drop(columns=['survived', 'passenger_id', 'sex', 'embarked', 'age'])
y_train = train.survived

X_validate = validate.drop(columns=['survived', 'passenger_id', 'sex', 'embarked','age'])
y_validate = validate.survived

X_test = test.drop(columns=['survived', 'passenger_id', 'sex', 'embarked','age'])
y_test = test.survived

1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [16]:
def establish_baseline(y_train):

    baseline_prediction = y_train.mode()

    y_train_pred = pd.Series((baseline_prediction[0]), range(len(y_train)))

    cm = confusion_matrix(y_train, y_train_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return accuracy

In [19]:
establish_baseline(y_train)

0.6161048689138576

Baseline prediction is: did not survive

2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [43]:
clf = DecisionTreeClassifier(max_depth=3)

In [44]:
clf = clf.fit(X_train, y_train)

In [46]:
y_pred = clf.predict(X_train)
y_pred

array([1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [47]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.3968254 , 0.6031746 ],
       [0.08695652, 0.91304348],
       [0.01724138, 0.98275862],
       [0.9125    , 0.0875    ],
       [0.8375    , 0.1625    ]])

3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [49]:
cm = confusion_matrix(y_train, y_pred)
cm

array([[299,  30],
       [ 68, 137]])

In [50]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86       329
           1       0.82      0.67      0.74       205

    accuracy                           0.82       534
   macro avg       0.82      0.79      0.80       534
weighted avg       0.82      0.82      0.81       534



In [51]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on validate set: 0.82


4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [52]:
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn)/(tn + fp + fn + tp)

true_positive_rate = tp/(tp + fn)
false_positive_rate = fp/(fp + tn)
true_negative_rate = tn/(tn + fp)
false_negative_rate = fn/(fn + tp)

precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*(precision*recall)/(precision+recall)

support_pos = tp + fn
support_neg = fp + tn

dict = {
    'metric' : ['accuracy'
                ,'true_positive_rate'
                ,'false_positive_rate'
                ,'true_negative_rate'
                ,'false_negative_rate'
                ,'precision'
                ,'recall'
                ,'f1_score'
                ,'support_pos'
                ,'support_neg']
    ,'score' : [accuracy
                ,true_positive_rate
                ,false_positive_rate
                ,true_negative_rate
                ,false_negative_rate
                ,precision
                ,recall
                ,f1_score
                ,support_pos
                ,support_neg]
}

pd.DataFrame(dict)

,metric,score
0,accuracy,0.816479
1,true_positive_rate,0.668293
2,false_positive_rate,0.091185
3,true_negative_rate,0.908815
4,false_negative_rate,0.331707
5,precision,0.820359
6,recall,0.668293
7,f1_score,0.736559
8,support_pos,205.000000
9,support_neg,329.000000


5. Run through steps 2-4 using a different max_depth value.

In [53]:
clf = DecisionTreeClassifier(max_depth=5)

In [54]:
clf = clf.fit(X_train, y_train)

In [55]:
y_pred = clf.predict(X_train)
y_pred[:10]

array([1, 1, 1, 0, 0, 1, 1, 0, 0, 0])

In [56]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.19230769, 0.80769231],
       [0.03571429, 0.96428571],
       [0.        , 1.        ],
       [0.90647482, 0.09352518],
       [0.76923077, 0.23076923]])

In [57]:
cm = confusion_matrix(y_train, y_pred)
cm

array([[306,  23],
       [ 62, 143]])

In [58]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88       329
           1       0.86      0.70      0.77       205

    accuracy                           0.84       534
   macro avg       0.85      0.81      0.82       534
weighted avg       0.84      0.84      0.84       534



In [59]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on validate set: 0.80


In [60]:
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn)/(tn + fp + fn + tp)

true_positive_rate = tp/(tp + fn)
false_positive_rate = fp/(fp + tn)
true_negative_rate = tn/(tn + fp)
false_negative_rate = fn/(fn + tp)

precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*(precision*recall)/(precision+recall)

support_pos = tp + fn
support_neg = fp + tn

dict = {
    'metric' : ['accuracy'
                ,'true_positive_rate'
                ,'false_positive_rate'
                ,'true_negative_rate'
                ,'false_negative_rate'
                ,'precision'
                ,'recall'
                ,'f1_score'
                ,'support_pos'
                ,'support_neg']
    ,'score' : [accuracy
                ,true_positive_rate
                ,false_positive_rate
                ,true_negative_rate
                ,false_negative_rate
                ,precision
                ,recall
                ,f1_score
                ,support_pos
                ,support_neg]
}

pd.DataFrame(dict)

,metric,score
0,accuracy,0.840824
1,true_positive_rate,0.697561
2,false_positive_rate,0.069909
3,true_negative_rate,0.930091
4,false_negative_rate,0.302439
5,precision,0.861446
6,recall,0.697561
7,f1_score,0.770889
8,support_pos,205.000000
9,support_neg,329.000000


6. Which model performs better on your in-sample data?

The depth 5 model performs better on the in-sample data 

7. Which model performs best on your out-of-sample data, the validate set?

The 3 depth model performs better on the validate data and there is a smaller difference between test and validate performance

### Part 2

1. Work through these same exercises using the Telco dataset.

In [4]:
telco_df = get_telco_data()
telco_df.head().T

,0,1,2,3,4
payment_type_id,2,2,1,1,2
internet_service_type_id,1,1,2,2,2
contract_type_id,2,1,1,1,1
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ
gender,Female,Male,Male,Male,Female
senior_citizen,0,0,0,1,1
partner,Yes,No,No,Yes,Yes
dependents,Yes,No,No,No,No
tenure,9,9,4,13,3
phone_service,Yes,Yes,Yes,Yes,Yes


In [5]:
def clean_telco(df):

    dummy_df = pd.get_dummies(df[['gender','partner', 'dependents', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing', 'contract_type', 'internet_service_type', 'payment_type']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    
    df = df.drop(columns =['payment_type_id','internet_service_type_id','contract_type_id', 'total_charges'])
    
    
    return df

In [6]:
telco_df = clean_telco(telco_df)
telco_df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,online_backup,...,streaming_movies_No internet service,streaming_movies_Yes,paperless_billing_Yes,contract_type_One year,contract_type_Two year,internet_service_type_Fiber optic,internet_service_type_None,payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,0002-ORFBO,Female,0,Yes,Yes,9,Yes,No,No,Yes,...,0,0,1,1,0,0,0,0,0,1
1,0003-MKNFE,Male,0,No,No,9,Yes,Yes,No,No,...,0,1,0,0,0,0,0,0,0,1
2,0004-TLHLJ,Male,0,No,No,4,Yes,No,No,No,...,0,0,1,0,0,1,0,0,1,0
3,0011-IGKFF,Male,1,Yes,No,13,Yes,No,No,Yes,...,0,1,1,0,0,1,0,0,1,0
4,0013-EXCHZ,Female,1,Yes,No,3,Yes,No,No,No,...,0,0,1,0,0,1,0,0,0,1


In [7]:
telco_df.head().T

,0,1,2,3,4
customer_id,0002-ORFBO,0003-MKNFE,0004-TLHLJ,0011-IGKFF,0013-EXCHZ
gender,Female,Male,Male,Male,Female
senior_citizen,0,0,0,1,1
partner,Yes,No,No,Yes,Yes
dependents,Yes,No,No,No,No
tenure,9,9,4,13,3
phone_service,Yes,Yes,Yes,Yes,Yes
multiple_lines,No,Yes,No,No,No
online_security,No,No,No,No,No
online_backup,Yes,No,No,Yes,No


In [8]:
train, validate, test = split_data(telco_df, col_to_stratify='churn')
train.shape, validate.shape, test.shape

((4225, 44), (1409, 44), (1409, 44))

In [9]:
X_train = train.drop(columns=['customer_id', 'churn', 'gender', 'partner', 'dependents', 'contract_type', 'internet_service_type', 'payment_type', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing'])
y_train = train.churn

X_validate = validate.drop(columns=['customer_id', 'churn', 'gender', 'partner', 'dependents', 'contract_type', 'internet_service_type', 'payment_type', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing'])
y_validate = validate.churn

X_test = test.drop(columns=['customer_id', 'churn', 'gender', 'partner', 'dependents', 'contract_type', 'internet_service_type', 'payment_type', 'phone_service', 'multiple_lines', 'online_security', 'online_backup', 'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'paperless_billing'])
y_test = test.churn



In [10]:
X_train.dtypes



senior_citizen                            int64
tenure                                    int64
monthly_charges                         float64
gender_Male                               uint8
partner_Yes                               uint8
dependents_Yes                            uint8
phone_service_Yes                         uint8
multiple_lines_No phone service           uint8
multiple_lines_Yes                        uint8
online_security_No internet service       uint8
online_security_Yes                       uint8
online_backup_No internet service         uint8
online_backup_Yes                         uint8
tech_support_No internet service          uint8
tech_support_Yes                          uint8
streaming_tv_No internet service          uint8
streaming_tv_Yes                          uint8
streaming_movies_No internet service      uint8
streaming_movies_Yes                      uint8
paperless_billing_Yes                     uint8
contract_type_One year                  

In [11]:
def establish_baseline(y_train):

    baseline_prediction = y_train.mode()

    y_train_pred = pd.Series((baseline_prediction[0]), range(len(y_train)))

    cm = confusion_matrix(y_train, y_train_pred)
    tn, fp, fn, tp = cm.ravel()

    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return accuracy

In [12]:
establish_baseline(y_train)

0.7346745562130178

In [13]:
clf = DecisionTreeClassifier(max_depth=3)

In [14]:
clf = clf.fit(X_train, y_train)

In [15]:
y_pred = clf.predict(X_train)
y_pred[:10]

array(['No', 'No', 'No', 'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'No'],
      dtype=object)

In [16]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.62555066, 0.37444934],
       [0.8026534 , 0.1973466 ],
       [0.62555066, 0.37444934],
       [0.12413793, 0.87586207],
       [0.54304636, 0.45695364]])

In [17]:
cm = confusion_matrix(y_train, y_pred)
cm

array([[2891,  213],
       [ 680,  441]])

In [18]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

          No       0.81      0.93      0.87      3104
         Yes       0.67      0.39      0.50      1121

    accuracy                           0.79      4225
   macro avg       0.74      0.66      0.68      4225
weighted avg       0.77      0.79      0.77      4225



In [19]:
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))

Accuracy of Decision Tree classifier on validate set: 0.80


In [20]:
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn)/(tn + fp + fn + tp)

true_positive_rate = tp/(tp + fn)
false_positive_rate = fp/(fp + tn)
true_negative_rate = tn/(tn + fp)
false_negative_rate = fn/(fn + tp)

precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1_score = 2*(precision*recall)/(precision+recall)

support_pos = tp + fn
support_neg = fp + tn

dict = {
    'metric' : ['accuracy'
                ,'true_positive_rate'
                ,'false_positive_rate'
                ,'true_negative_rate'
                ,'false_negative_rate'
                ,'precision'
                ,'recall'
                ,'f1_score'
                ,'support_pos'
                ,'support_neg']
    ,'score' : [accuracy
                ,true_positive_rate
                ,false_positive_rate
                ,true_negative_rate
                ,false_negative_rate
                ,precision
                ,recall
                ,f1_score
                ,support_pos
                ,support_neg]
}

pd.DataFrame(dict)

,metric,score
0,accuracy,0.788639
1,true_positive_rate,0.393399
2,false_positive_rate,0.068621
3,true_negative_rate,0.931379
4,false_negative_rate,0.606601
5,precision,0.674312
6,recall,0.393399
7,f1_score,0.496901
8,support_pos,1121.000000
9,support_neg,3104.000000


2. Experiment with this model on other datasets with a higher number of output classes.